In [176]:
import pandas as pd
import itertools
import numpy as np
from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

In [170]:
df = pd.read_excel('/Users/Mercer/Desktop/data40+E.81896.xlsx')
df

,ds,y,code
0,2018-6-11,150,PM081896
1,2018-6-18,10,PM081896
2,2018-6-25,45,PM081896
3,2018-7-16,100,PM081896
4,2018-7-23,220,PM081896
...,...,...,...
57,2019-3-4,25,PM081896
58,2019-3-11,10,PM081896
59,2019-2-4,209,PM081896
60,2019-2-11,95,PM081896


In [171]:
# Prophet的参数需要在这里调整
def pht(dataframe):
    
    df_train = dataframe[dataframe['ds']<='2019-12-31']
    df_test = dataframe[dataframe['ds']>'2019-12-31']
    
    param_grid = {  
        'changepoint_prior_scale': [0.001, 0.01, 0.1],
        'seasonality_prior_scale': [0.01, 0.1, 1.0],
    }
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    rmses = []
    
    df['cap'] = 20000000
    df['floor'] = 0
    
    for params in all_params:
        m = Prophet(**params).fit(df_train) 
        df_cv = cross_validation(m, horizon='120 days', parallel="processes")   
        df_p = performance_metrics(df_cv, rolling_window=1)
        rmses.append(df_p['rmse'].values[0])

    best_params = all_params[np.argmin(rmses)]
    m = Prophet(**best_params).fit(df_train)
    future = m.make_future_dataframe(periods=750)
    
    future['cap'] = 20000000
    future['floor'] = 0
    
    forecast = m.predict(future)
    
    oo = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
    oo.insert(oo.shape[1],'index',df.loc[i-1,'code'])
    
    return oo

In [173]:
# 预设空的dataframe，最后导出的就是这个
result = pd.DataFrame()

result

""


In [174]:
# 这是行数
k = range(len(df))
print(k)

range(0, 62)


In [175]:
# loop部分
temp = pd.DataFrame()
for i in k:
    if i != 0 and df.loc[i,'code'] != df.loc[i-1,'code']:
        temp_fc = pht(temp)
        result = result.append(temp_fc)
        temp = pd.DataFrame()
    else:
        temp = temp.append(df.loc[i,:])

temp_fc = pht(temp)
result = result.append(temp_fc)
        
result

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 19.
INFO:prophet:Making 1 forecasts with cutoffs between 2019-08-04 00:00:00 and 2019-08-04 00:00:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7ff0b9496820>
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 19.
INFO:prophet:Making 1 forecasts 

,ds,yhat,yhat_lower,yhat_upper,index
0,2018-06-11,112.993732,35.282875,200.246301,PM081896
1,2018-06-18,112.510302,29.620564,198.550180,PM081896
2,2018-06-25,112.026871,21.538605,196.916484,PM081896
3,2018-07-16,110.576580,25.270269,192.237167,PM081896
4,2018-07-23,110.093150,23.468680,190.286276,PM081896
...,...,...,...,...,...
771,2021-12-17,24.249729,-62.276877,112.436596,PM081896
772,2021-12-18,24.180668,-56.673624,106.791921,PM081896
773,2021-12-19,24.111606,-59.268478,112.365768,PM081896
774,2021-12-20,24.042545,-56.156537,119.477542,PM081896


In [177]:
# 导出数据
result.to_excel('/Users/Mercer/Desktop/result40+E.81899.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/Mercerx/Desktop/result40+E.81899.xlsx'